# Annotation task submission notebook

## Init libraries

In [2]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])
project = 'd3b-bixu/dev-wgsa'

## Draft task def

In [ ]:
def draft_task(task_name, input_dict, app_name, project):
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
    task.inputs['output_basename'] = task.id
    task.save()


## Run task def

In [ ]:
def run_tasks(project, prefix):
    # set prefix to all if you don't need to be selective
    draft_tasks = list(api.tasks.query(project=project, status='DRAFT').all())
    for i in range(0, len(draft_tasks), 1):
        if prefix == 'ALL' or re.search(prefix, draft_tasks[i].name):
            draft_tasks[i].run()
            print('Running task ' + draft_tasks[i].id + ' ' + draft_tasks[i].name)
        else:
            print('Task ' + draft_tasks[i].id + ' ' + draft_tasks[i].name + ' skipped, prefix ' + prefix + ' did not match')


### execute run tasks

In [ ]:
prefix = 'KF ANNOT UNK VAR'
print("You sure you want to run all tasks with prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    run_tasks(project, prefix)
else:
    sys.stderr.write("User did not type YASS, skipping\n")

## Remove old annotation task set up

In [ ]:
manifest = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/original_trio_vcf_test-manifest.csv')
head = next(manifest)
app = project + '/bcftools-strip-info'
strip_info = 'INFO/ANN'
tool_name = 'gatk.denovo.trio'
for line in manifest:
    info = line.split(',')
    in_dict = {}
    in_dict['input_vcf'] = api.files.get(info[0])
    in_dict['tool_name'] = tool_name
    in_dict['strip_info'] = strip_info
    task_name = "BCFTOOLS STRIP ANNO: " + in_dict['input_vcf'].metadata['Kids First Family ID']
    draft_task(task_name, in_dict, app, project)
    

## Copy metadata to outputs

In [ ]:
def add_metadata_to_outputs(task, phrase, in_key):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[in_key].metadata
        for out_key in task.outputs:
            # pdb.set_trace()
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        if type(output) is not list:
                            file_obj = api.files.get(output.id)
                            for key in metadata:
                                file_obj.metadata[key] = metadata[key]
                            file_obj.save()
                        else:
                            for item in output:
                                if item is not None:
                                    file_obj = api.files.get(item.id)
                                    for key in metadata:
                                        file_obj.metadata[key] = metadata[key]
                                    file_obj.save()

            except Exception as e:
                print(e)
                print("Skipping " + out_key + " for " + task.name + " due to error")

#### Add metadata to file normal tasks

In [ ]:
prefix = 'KF ANNOT UNK VAR'
key = 'input_vcf'
print("You sure tag outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    for task in tasks:
        add_metadata_to_outputs(task, prefix, key)
else:
    sys.stderr.write("User did not type YASS, skipping\n")

### Add file tag to task outputs

In [ ]:
def tag_file_outputs(task, phrase, tags):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        for out_key in task.outputs:
            # pdb.set_trace()
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    file_obj.tags = tags
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        if type(output) is not list:
                            file_obj = api.files.get(output.id)
                            file_obj.tags = tags
                            file_obj.save()
                        else:
                            for item in output:
                                if item is not None:
                                    file_obj = api.files.get(item.id)
                                    file_obj.tags = tags
                                    file_obj.save()

            except Exception as e:
                print(e)
                print("Skipping " + out_key + " for " + task.name + " due to error")

In [ ]:
prefix = 'MARAZITA CALLER ONLY TEST'
tags = ['ANNOTATED', 'CALLER_ONLY_ALL_VAR']
print("You sure tag outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    for task in tasks:
        tag_file_outputs(task, prefix, tags)
else:
    sys.stderr.write("User did not type YASS, skipping\n")

#### Tag batch tasks

In [ ]:
prefix = 'bcftools-filter-vcf run - RM SNP'
key = 'input_vcf'
print("You sure tag outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    for task in tasks:
        if re.search(prefix, task.name):
            for child in task.get_batch_children():
                tag_outputs(child, prefix, key)
else:
    sys.stderr.write("User did not type YASS, skipping\n")

#### Delete task outputs

In [ ]:
prefix = 'WGSA SNP INDEL ALL'
print("You sure you want to delete outputs from tasks with prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    for task in tasks:
        try:
            if re.search(prefix, task.name):
                for key in task.outputs:
                    if isinstance(task.outputs[key], list):
                        for f_obj in task.outputs[key]:
                            f_obj.delete()
                    elif task.outputs[key] is not None:
                        task.outputs[key].delete()
        except Exception as e:
            sys.stderr.write(str(e) + "\nError processing " + task.name + "task id: " + task.id + ". Review error if ok\n")


## Annotation runs

In [ ]:
stripped_vcf_manifest = '/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/stripped_vcf-manifest.csv'
snp_rm_vcf_manifest = '/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/snp_rm-manifest.csv'
subset_test = '/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/subset_test-manifest.csv'

### Set up snpEff run

In [ ]:
def get_snpEff_refs(reference_name, tool_name):
    ref_dict = {}
    ref_dict['reference_name'] = reference_name
    ref_dict['ref_tar_gz'] = api.files.query(project=project, names=['snpeff_hg38_grch38.tgz'])[0]
    ref_dict['tool_name'] = tool_name
    # db file name list has optional databases to run
#     if len(vcf_list) > 0:
#         ref_dict['db_vcfs'] = []
#         for vcf in vcf_list:
#             ref_dict['db_vcfs'].append(api.files.query(project=project, names=[vcf])[0])
#     if gwas_bool:
#         ref_dict['gwas_catalog_txt'].append(api.files.query(project=project, names=['gwas_catalog_v1.0-associations_e98_r2020-03-08.tsv'])[0])
#     if dbnsfp_txt_bool:
#         ref_dict['dbnsfp_txt'].append(api.files.query(project=project, names=['dbNSFP4.0a.gz'])[0])
    return ref_dict


In [ ]:
# check all vars
app = project + "/snpeff-annotate"
manifest = open(snp_rm_vcf_manifest)
task_prefix = 'snpEff NO DB NO SNP refGene: '
# hg38 or GRCh38.86
ref_gene_model = "hg38"
# run gwas?
# gwas_bool = False
# run dbnsfp?
# dbnsfp_txt_bool = False
tool_name = "gatk.denovo.trio.stripped"

ref_obj = get_snpEff_refs(ref_gene_model, tool_name)
head = next(manifest)
for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)


### Set up annovar run

In [ ]:
def get_ANNOVAR_refs(db_list, db_run_bool, protocol_name):
    ref_dict = {}
    ref_dict['run_dbs'] = db_run_bool
    ref_dict['cache'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['protocol_name'] = protocol_name
    # db file name list has optional databases to run
    if len(db_list) > 0:
        ref_dict['additional_dbs'] = []
        for db in db_list:
            ref_dict['additional_dbs'].append(api.files.query(project=project, names=[db])[0])
    return ref_dict


In [ ]:
app = project + "/kfdrc-annovar"
manifest = open(snp_rm_vcf_manifest)
task_prefix = 'ANNOVAR NO DB NO SNP refGene: '
additional_dbs = ['esp6500siv2_all.tgz']
db_run_bool = False
# choices are refGene, ensGene, knownGene
protocol_name = 'refGene'


ref_obj = get_ANNOVAR_refs(additional_dbs, db_run_bool, protocol_name)
head = next(manifest)
for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)


### Set up VEP run

In [ ]:
def get_VEP_refs(db_key_list, db_run_bool):
    extra_db_dict = {'cadd_indels': 'CADDv1.5-38-InDels.tsv.gz', 'cadd_snvs': 'CADDv1.5-38-whole_genome_SNVs.tsv.gz',
                     'dbnsfp': 'dbNSFP4.0a.gz', 'dbscsnv': 'dbscSNV1.1_GRCh38.txt.gz', 'phylop': 'hg38.phyloP100way.bw'}
    ref_dict = {}
    ref_dict['run_cache_dbs'] = db_run_bool
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens.GRCh38.dna.toplevel.fa.gz'])[0]
    ref_dict['cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['tool_name'] = 'VEP99'
    # db_key list has optional databases to run
    for key in db_key_list:
        ref_dict[key] = api.files.query(project=project, names=[extra_db_dict[key]])[0]
    return ref_dict
    

In [ ]:
app = project + "/kfdrc-vep99-wgsa"
manifest = open(snp_rm_vcf_manifest)
task_prefix = 'VEP NO DB NO SNP: '
additional_dbs = []
db_run_bool = False

ref_obj = get_VEP_refs(additional_dbs, db_run_bool)
head = next(manifest)
for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)


### Set up WGSA run

In [ ]:
def get_WGSA_refs(settings, tool_name, db_list):
    ref_dict = {}
    # db_list = ['dbSNP.tgz', 'GWAS_catalog.tgz', 'wgsa_hg38_resource.tgz', '1000Gp3.tgz', 'UK10K.tgz', 'ESP6500.tgz', 'ExACr0.3.tgz',
#                'dbNSFP.tgz', 'CADDv1.4.tgz', 'clinvar.tgz', 'wgsa_hg19_resource.tgz', 'COSMIC_hg38.tgz', 'PhyloP_hg38.tgz', 'gnomAD.tgz',
#                'crossmap.tgz']
    ref_dict['tool_name'] = tool_name
    ref_dict['annovar_ref'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['snpeff_ref'] = api.files.query(project=project, names=['snpeff_hg38_grch38.tgz'])[0]
    ref_dict['VEP_cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens.GRCh38.dna.toplevel.fa.gz'])[0]
    ref_dict['settings'] = api.files.query(project=project, names=[settings])[0]
    # db file name list has optional databases to run
    if len(db_list) > 0:
        ref_dict['resources'] = []
        for db in db_list:
            ref_dict['resources'].append(api.files.query(project=project, names=[db])[0])
    return ref_dict


In [ ]:
app = project + "/kfdrc-wgsa-annotate"
manifest = open(stripped_vcf_manifest)
task_prefix = 'WGSA SNP INDEL ALL RPT: '
tool_name = "gatk.denovo.trio.stripped.all_annot"
settings = "wgsa_all_desired_settings.txt"
# settings = "WGSA_indel_only_ALL.txt"
db_list = ['precomputed_hg38.tgz', 'dbSNP.tgz', 'GWAS_catalog.tgz', 'wgsa_hg38_resource.tgz', '1000Gp3.tgz', 'UK10K.tgz', 'ESP6500.tgz', 'ExACr0.3.tgz',
           'dbNSFP.tgz', 'CADDv1.4.tgz', 'clinvar.tgz', 'wgsa_hg19_resource.tgz', 'COSMIC_hg38.tgz', 'PhyloP_hg38.tgz', 'gnomAD.tgz',
           'crossmap.tgz']

ref_obj = get_WGSA_refs(settings, tool_name, db_list)
head = next(manifest)

for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)


### Set up all caller + db run

In [ ]:
def get_caller_db_refs(tool_name, strip_info, filter_vcf):
    ref_dict = {}
    ref_dict['tool_name'] = tool_name
    ref_dict['strip_info'] = strip_info
    if filter_vcf:
        ref_dict['include_expression'] = filter_vcf
    ref_dict['ANNOVAR_cache'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['ANNOVAR_cosmic_db'] = api.files.query(project=project, names=['cosmic90.tgz'])[0]
    ref_dict['ANNOVAR_dbscsnv_db'] = api.files.query(project=project, names=['dbscsnv11.tgz'])[0]
    ref_dict['ANNOVAR_kg_db'] = api.files.query(project=project, names=['1000g2015aug.tgz'])[0]
    ref_dict['ANNOVAR_esp_db'] = api.files.query(project=project, names=['esp6500siv2_all.tgz'])[0]
    ref_dict['ANNOVAR_gnomad_db'] = api.files.query(project=project, names=['gnomad30_genome.tgz'])[0]
    ref_dict['ANNOVAR_run_dbs_refGene'] = True
    ref_dict['ANNOVAR_run_dbs_ensGene'] = False
    ref_dict['ANNOVAR_run_dbs_knownGene'] = False
    ref_dict['snpEff_ref_tar_gz'] = api.files.query(project=project, names=['snpeff_hg38_grch38.tgz'])[0]
    ref_dict['gwas_cat_db_file'] = api.files.query(project=project, names=['gwas_catalog_v1.0-associations_e98_r2020-03-08.tsv'])[0]
    ref_dict['SnpSift_vcf_db_name'] = "ClinVar"
    ref_dict['SnpSift_vcf_fields'] = "AF_ESP,AF_EXAC,AF_TGP,ALLELEID,CLNDN,CLNDNINCL,CLNDISDB,CLNDISDBINCL,CLNHGVS,CLNREVSTAT,CLNSIG,CLNSIGCONF,CLNSIGINCL,CLNVC,CLNVCSO,CLNVI,DBVARID,GENEINFO,MC,ORIGIN,RS,SSR"
    ref_dict['clinvar_vcf'] = api.files.query(project=project, names=['clinvar-2020-03-17.vcf.gz'])[0]
    ref_dict['VEP_cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens.GRCh38.dna.toplevel.fa.gz'])[0]
    ref_dict['VEP_run_cache_existing'] = True
    ref_dict['VEP_run_cache_af'] = True
    ref_dict['VEP_cadd_indels'] = api.files.query(project=project, names=['CADDv1.5-38-InDels.tsv.gz'])[0]
    ref_dict['VEP_cadd_snvs'] = api.files.query(project=project, names=['CADDv1.5-38-whole_genome_SNVs.tsv.gz'])[0]
    ref_dict['VEP_dbnsfp'] = api.files.query(project=project, names=['dbNSFP4.0a.gz'])[0]
    return ref_dict

In [ ]:
app = project + "/kf-caller-db-wf"
manifest = open(subset_test)
task_prefix = 'ALL CALLER W DB NO SNP TEST: '
tool_name = "gatk.denovo.trio"
strip_info = "INFO/ANN"
filter_vcf = "TYPE!=\"snp\""
# db_list = ['cosmic90.tgz', 'dbscsnv11.tgz','gnomad30_genome.tgz']
ref_obj = get_caller_db_refs(tool_name, strip_info, filter_vcf)
head = next(manifest)

for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)


### Set up caller only run

In [ ]:
def get_caller_only_refs(tool_name, strip_info, filter_vcf):
    ref_dict = {}
    ref_dict['tool_name'] = tool_name
    ref_dict['strip_info'] = strip_info
    if filter_vcf:
        ref_dict['include_expression'] = filter_vcf
    ref_dict['ANNOVAR_cache'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['ANNOVAR_run_dbs_refGene'] = False
    ref_dict['ANNOVAR_run_dbs_ensGene'] = False
    ref_dict['ANNOVAR_run_dbs_knownGene'] = False
    ref_dict['snpEff_ref_tar_gz'] = api.files.query(project=project, names=['snpeff_hg38_grch38.tgz'])[0]
    ref_dict['VEP_cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens.GRCh38.dna.toplevel.fa.gz'])[0]
    ref_dict['VEP_run_cache_existing'] = False
    ref_dict['VEP_run_cache_af'] = False
    return ref_dict

In [ ]:
app = project + "/kf-caller-only-wf"
manifest = open(subset_test)
task_prefix = 'ALL CALLER ONLY NO SNP TEST: '
tool_name = "gatk.denovo.trio"
strip_info = "INFO/ANN"
filter_vcf = "TYPE!=\"snp\""
ref_obj = get_caller_only_refs(tool_name, strip_info, filter_vcf)
head = next(manifest)

for line in manifest:
    info = line.split(',')
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    in_vcf = api.files.get(info[0])
    task_name = task_prefix + in_vcf.metadata['Kids First Family ID']
    in_dict['input_vcf'] = in_vcf
    draft_task(task_name, in_dict, app, project)

#### simple utility to expand ipython view in browser

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

## Get Task Cost/Run analysis

In [ ]:
def merge_and_sum_time_intervals(task):
    data = []
    for job in task.get_execution_details().jobs:
        if job.status != "COMPLETED":
            sys.stderr.write("Skipping job likely killed due to spot instance kill for " + job.name + " from task " + task.id + "\n")
        else:
            try:
                pair = (job.start_time, job.end_time)
                data.append(pair)
            except Exception as e:
                print (e)
                pdb.set_trace()
                hold = 1
    data = sorted(data, key=lambda x: x[0])


    # from https://stackoverflow.com/questions/34797525/how-to-correctly-merge-overlapping-datetime-ranges-in-python
    result = []
    try:
        t_old = data[0]
        for t in data[1:]:
            if t_old[1] >= t[0]:  #I assume that the data is sorted already
                t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
            else:
                result.append(t_old)
                t_old = t

        else:
            result.append(t_old)
    except Exception as e:
        print(e)
        pdb.set_trace()
        hold = 1
    total_seconds = 0
    for t_int in result:
        total_seconds += (t_int[1] - t_int[0]).seconds
    #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
    return [task.id, task.name, str(total_seconds/3600)]


In [ ]:
def get_cost(task):
    try:
        return [task.id, task.name, str(task.price.amount), task.app]
    except Exception as e:
        sys.stderr.write('Could not process task ' + task.id + ' ' + task.name + ' got error ' + str(e) + '\n')
        sys.exit()

In [ ]:
task_prefix = "^WGSA"
task_summary = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/WGSA_test,txt', 'w')
task_summary.write("task_id\ttask_name\trun_hrs\tcost\tworkflow\n")
for task in api.tasks.query(project=project, status="COMPLETED").all():
    if re.search(task_prefix, task.name):
        (rid, rname, duration) = merge_and_sum_time_intervals(task)
        (cid, cname, cost, wf) = get_cost(task)
        task_summary.write("\t".join([rid, rname, duration, cost, wf]) + "\n")
task_summary.close()

## Get all task outputs

In [3]:
def write_to_manifest(out_fh, file_obj, out_key, task_name):
    out_fh.write(",".join([file_obj.id, file_obj.name, out_key, task_name]) + "\n")

In [4]:
#test= api.files.query(project=project, metadata={'task_id':'fdb0d85a-2005-4d3e-a3cf-43fb735d13de'}).all()
tasks = api.tasks.query(project=project, status="COMPLETED").all()
out = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/EXPORT/unk_indel_vcf_manifest.csv', 'w')
out.write("id,name,output_category,task_name\n")
phrase = "KF ANNOT UNK VAR"
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Processing task: ' + task.name + "\n")
        for out_key in task.outputs:
#             try:
            if type(task.outputs[out_key]) is not list:
                file_obj = task.outputs[out_key]
                write_to_manifest(out, file_obj, out_key, task.name)
                if task.outputs[out_key].secondary_files is not None:
                    write_to_manifest(out, task.outputs[out_key].secondary_files[0], out_key, task.name)
            else:
                for i in range(len(task.outputs[out_key])):
                    if type(task.outputs[out_key][i]) is not list:
                        write_to_manifest(out, task.outputs[out_key][i], out_key, task.name)
                        if task.outputs[out_key][i].secondary_files is not None:
                            write_to_manifest(out, task.outputs[out_key][i].secondary_files[0], out_key, task.name)
                    else:
                        for j in range(len(task.outputs[out_key][i])):
                            if task.outputs[out_key][i][j] is not None:
                                write_to_manifest(out, task.outputs[out_key][i][j], out_key, task.name)
                                if task.outputs[out_key][i][j].secondary_files is not None:
                                    write_to_manifest(out, task.outputs[out_key][i][j].secondary_files[0], out_key, task.name)
#             except Exception as e:
#                 print(e)
#                 print("Skipping " + out_key + " for " + task.name + " due to error")
out.close()


Processing task: KF ANNOT UNK VAR: FM_CN3QK5A1
Processing task: KF ANNOT UNK VAR: FM_DFW18WG8
Processing task: KF ANNOT UNK VAR: FM_0ADXQA3J
Processing task: KF ANNOT UNK VAR: FM_0DB3211J
Processing task: KF ANNOT UNK VAR: FM_ZFMCW3G3
Processing task: KF ANNOT UNK VAR: FM_SHJNRP8S
Processing task: KF ANNOT UNK VAR: FM_4Y21X6PP
Processing task: KF ANNOT UNK VAR: FM_0FJQBB97
Processing task: KF ANNOT UNK VAR: FM_RT6VGNSJ
Processing task: KF ANNOT UNK VAR: FM_1PCY4Y30
Processing task: KF ANNOT UNK VAR: FM_V9TXBYS6
Processing task: KF ANNOT UNK VAR: FM_11ASN8XN
Processing task: KF ANNOT UNK VAR: FM_DKX82ZD7
Processing task: KF ANNOT UNK VAR: FM_3FWV6NG5
Processing task: KF ANNOT UNK VAR: FM_ZETA86QZ
Processing task: KF ANNOT UNK VAR: FM_PR07SNT9
Processing task: KF ANNOT UNK VAR: FM_2C8A094S
Processing task: KF ANNOT UNK VAR: FM_0RWXQH9X
Processing task: KF ANNOT UNK VAR: FM_NH1CE0E4
Processing task: KF ANNOT UNK VAR: FM_E6EA6D9M
Processing task: KF ANNOT UNK VAR: FM_1046D86B
Processing ta

### Add file metadata to manifest

In [ ]:
manifest = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/EXPORT/faux_vcf_manifest.csv')
head = next(manifest)
head = head.rstrip('\n')
out = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/EXPORT/exported_vcfs.csv', 'w')
out.write(head)

init = next(manifest)
info = init.rstrip('\n').split(',')
fobj = api.files.get(info[0])
keys = []
temp = ','.join(info)
for key in fobj.metadata:
    temp += "," + fobj.metadata[key]
    keys.append(key)
#pdb.set_trace()
out.write("," + ",".join(keys) + "\n" + temp + "\n")
for line in manifest:
    info = line.rstrip('\n').split(',')
    out.write(",".join(info))
    fobj = api.files.get(info[0])
    if fobj.metadata is None:
        out.write(",,\n")
    else:
        for key in fobj.metadata:
            if key not in keys:
                sys.stderr.write("ERROR: " + key + " not in keys, rethink strategy for file " + fobj.name + "\n")
                pdb.set_trace()
                exit(1)
            else:
                out.write("," + fobj.metadata[key])
        out.write("\n")
out.close()

### Add file tags to taks outputs
Use output from [Get all task outputs](##-Get-all-task-outputs) and add a csv column with desired tags to add

In [5]:
manifest = open('/Users/brownm28/Documents/2020-Mar-4_WGSA/EXPORT/missed_tbi.txt')
head = next(manifest)
header = head.rstrip('\n').split('\t')
tag_col_name = "TAG_TO_ADD"
idx = header.index(tag_col_name)
print("You sure want to tag these files using this manifest? Type \"YASS\" if so")
check = input()
x = 1
m = 50
if check == "YASS":
    for line in manifest:
        if x % m == 0:
            sys.stderr.write("Processed " + str(x) + " files\n")
        info = line.rstrip('\n').split('\t')
        file_obj = api.files.get(info[0])
        try:
            tags = info[idx].split(',')
            file_obj.tags = tags
            file_obj.save()
        except Exception as e:
            sys.stderr.write(str(e) + "\nCould not tag file with ID: " + info[0] + ", skipping!\n")
        x += 1


You sure want to tag these files using this manifest? Type "YASS" if so
YASS


Processed 50 files


# Split WF Runs
For creating precomputed annotations on simulated SNPs and known indels

In [ ]:
def draft_split_task(task_name, input_dict, app_name, project, annotator):
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
    task.inputs['output_basename'] = task.id + "_" + annotator
    task.save()

### Run VEP Split

In [ ]:
def get_vep_split_refs(run_norm_bool, tool_name):
    ref_dict = {}
    # may need to change bed
    ref_dict['scatter_bed'] = api.files.query(project=project, names=['ucsc_indel_regions.bed'])[0]
#     UNCOMMENT BELOW If INPUT IS MASSIVE SNP FILE
#     ref_dict['scatter_ct'] = 200
#     ref_dict['bands'] = 1000000
#     ref_dict['ram'] = 72
#     ref_dict['cores'] = 36
    ref_dict['run_vt_norm'] = run_norm_bool
    ref_dict['tool_name'] = tool_name
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_dict['reference_dict'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dict'])[0]
    ref_dict['VEP_cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['VEP_buffer_size'] = 20000
    ref_dict['VEP_run_cache_af'] = False
    ref_dict['VEP_run_cache_existing'] = False
    ref_dict['VEP_run_stats'] = False
    return ref_dict



In [ ]:
in_vcf = api.files.get('5ec57ff9e4b05495c9a85be8')
task_name = "KFDRC VEP Precomputed: DBSNP V153 non snps"
run_norm_bool = False
tool_name = "DBSNP_SNP_RM_V153"
annotator = "VEP"
app_name = project + "/kf-vep99-split-sub-wf"
in_dict = {}
ref_objs = get_vep_split_refs(run_norm_bool, tool_name)
for key in ref_objs:
    in_dict[key] = ref_objs[key]
in_dict['input_vcf'] = in_vcf
draft_split_task(task_name, in_dict, app_name, project, annotator)



### Run snpEff Split

In [ ]:
def get_snpEff_refs(reference_name, tool_name):
    ref_dict = {}
    # may need to change bed
    ref_dict['scatter_bed'] = api.files.query(project=project, names=['ucsc_indel_regions.bed'])[0]
    ref_dict['snpeff_ref_name'] = ['hg38kg', 'hg38', 'GRCh38.86']
    ref_dict['snpEff_ref_tar_gz'] = api.files.query(project=project, names=['snpeff_hg38_hg38kg_grch38.tgz'])[0]
    ref_dict['wf_tool_name'] = tool_name
#     UNCOMMENT BELOW If INPUT IS MASSIVE SNP FILE
#     ref_dict['scatter_ct'] = 200
#     ref_dict['bands'] = 1000000
#     ref_dict['ram'] = 72
#     ref_dict['cores'] = 36
    ref_dict['run_vt_norm'] = run_norm_bool
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_dict['reference_dict'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dict'])[0]
    return ref_dict


In [ ]:
in_vcf = api.files.get('5ec57ff9e4b05495c9a85be8')
task_name = "KFDRC snpEff Precomputed: DBSNP V153 non snps"
run_norm_bool = False
tool_name = "DBSNP_SNP_RM_V153"
annotator = "snpEff"
app_name = project + "/kf-snp-eff-split-sub-wf"
in_dict = {}
ref_objs = get_snpEff_refs(run_norm_bool, tool_name)
for key in ref_objs:
    in_dict[key] = ref_objs[key]
in_dict['input_vcf'] = in_vcf
draft_split_task(task_name, in_dict, app_name, project, annotator)


### Run ANNOVAR Split

In [ ]:
def get_annovar_refs(reference_name, tool_name):
    ref_dict = {}
    # may need to change bed
    ref_dict['scatter_bed'] = api.files.query(project=project, names=['ucsc_indel_regions.bed'])[0]
    ref_dict['protocol_list'] = ['refGene', 'knownGene', 'ensGene']
    ref_dict['ANNOVAR_cache'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['wf_tool_name'] = tool_name
    ref_dict['run_dbs'] = [False]
#     UNCOMMENT BELOW If INPUT IS MASSIVE SNP FILE
#     ref_dict['scatter_ct'] = 200
#     ref_dict['bands'] = 1000000
#     ref_dict['ram'] = 128
#     ref_dict['cores'] = 40
    ref_dict['run_vt_norm'] = run_norm_bool
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_dict['reference_dict'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dict'])[0]
    return ref_dict


In [ ]:
in_vcf = api.files.get('5ec57ff9e4b05495c9a85be8')
task_name = "KFDRC ANNOVAR Precomputed: DBSNP V153 non snps"
run_norm_bool = False
tool_name = "DBSNP_SNP_RM_V153"
annotator = "ANNOVAR"
app_name = project + "/kf-annovar-w-preprocess-sub-wf"
in_dict = {}
ref_objs = get_annovar_refs(run_norm_bool, tool_name)
for key in ref_objs:
    in_dict[key] = ref_objs[key]
in_dict['input_vcf'] = in_vcf
draft_split_task(task_name, in_dict, app_name, project, annotator)


# Unknown indel annotation
Workflow to take a vcf, normalize it, filter on unkown indels, and annotate. Outputs are normalized vcf and annotated unknown indel results from ANNOVAR, VEP, snpEff

In [ ]:
def draft_unk_task(task_name, input_dict, app_name, project):
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
    task.inputs['output_basename'] = task.id
    task.save()

### Run unknown annotation

In [ ]:
def get_annot_refs(tool_name, reference_vcf, run_norm_bool, strip_info, include_expression):
    ref_dict = {}
    # general - required
    ref_dict['reference_vcf'] = reference_vcf
    ref_dict['tool_name'] = tool_name
    # general - optional
    if run_norm_bool:
        ref_dict['run_norm_flag'] = run_norm_bool
    if strip_info:
        ref_dict['strip_info'] = strip_info
    if include_expression:
        ref_dict['include_expression'] = include_expression
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    # ref_dict['reference_dict'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dict'])[0]
    # VEP specific
    ref_dict['VEP_cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['VEP_buffer_size'] = 20000
    ref_dict['VEP_run_cache_af'] = False
    ref_dict['VEP_run_cache_existing'] = False
    ref_dict['VEP_run_stats'] = False
    # ANNOVAR specific
    ref_dict['protocol_list'] = ['refGene', 'knownGene', 'ensGene']
    ref_dict['ANNOVAR_cache'] = api.files.query(project=project, names=['annovar_2019Oct24.tgz'])[0]
    ref_dict['ANNOVAR_run_dbs'] = [False]
    # snpEff specific
    ref_dict['snpeff_ref_name'] = ['hg38kg', 'hg38', 'GRCh38.86']
    ref_dict['snpEff_ref_tar_gz'] = api.files.query(project=project, names=['snpeff_hg38_hg38kg_grch38.tgz'])[0]
    return ref_dict



In [ ]:
in_vcf_manifest = '/Users/brownm28/Documents/2020-Mar-4_WGSA/TASK_RUN/unk_test_set-manifest.csv'
app_name = project + "/kf-unknown-var-annot-wf"
ref_vcf = api.files.query(project=project, names=['dbSNP_v153_ucsc-compatible.normalized.snps_rm.vcf.gz'])[0]
# change if somatic
tool_name = 'gatk'
run_norm = True
# for germline calls, probably INFO/ANN, somatic, probably INFO/CSQ
strip_info = "INFO/ANN"
include_expression = 'TYPE!="snp"'
vcf_list = open(in_vcf_manifest)
head = next(vcf_list)
header = head.rstrip('\n').split(',')
# get col of desired ID to ass to task name
id_label = "Kids First Family ID"
idx = header.index(id_label)
ref_objs = get_annot_refs(tool_name, ref_vcf, run_norm, strip_info, include_expression) 
for line in vcf_list:
    in_dict = {}
    # pdb.set_trace()
    for key in ref_objs:
        in_dict[key] = ref_objs[key]
    info = line.rstrip('\n').split(',')
    task_name = "KF ANNOT UNK VAR: " + info[idx]
    in_dict['input_vcf'] = api.files.get(info[0])
    draft_unk_task(task_name, in_dict, app_name, project)
    

In [ ]:
task = api.tasks.get('e3f66229-7217-4ec0-9d76-341b5a772226')
pdb.set_trace()
hold = 1